------
# Preporcess Population
##### 1. Assign Suburbs to SA2
##### 2. Find Subrub Population Proportion to SA2
##### 3. Predict Suburb Future Population and Density
-------

### 1. Assign Suburbs to SA2
1. Load datasets
2. Find out suburbs that belongs to 2 SA2
3. Assign Suburbs based on the area porportion
4. ...


load datasets


In [1]:
import pandas as pd
import geopandas as gpd

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
population = pd.read_csv("../data/raw/external_data/population.csv")
population.head()

,Unnamed: 0,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,SA2 name,ERP 2020,ERP 2021,ERP Change Amount,ERP Change Rate,Natureal increase,Internal migration,Overseas migration,Area km2,density 2021 persons/km2
0,0,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011001,Alfredton,15030,16263,1233,8.2,117,1143,-27,52.7,308.5
1,1,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011002,Ballarat,12097,11931,-166,-1.4,-43,-107,-16,12.4,963.8
2,2,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011003,Ballarat - North,24606,24720,114,0.5,8,123,-17,92.4,267.7
3,3,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011004,Ballarat - South,25639,25543,-96,-0.4,83,-150,-29,32.9,777.5
4,4,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011005,Buninyong,7721,7668,-53,-0.7,33,-78,-8,51.6,148.6


In [3]:
properties= pd.read_csv("../data/raw/properties_preporcessed_1.csv")
properties.head()

,Unnamed: 0,name,cost_text,rooms,type,cloest station,station_distance,cloest school,school_distance,CBD_distance,...,text,beds,bath,parking,Longitude,Latitude,SA2_CODE21,LOC_PID,LOC_NAME,SA2_MAIN16
0,0,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,"['3 Beds', '2 Baths', '2 Parking']",Apartment / Unit / Flat,30,1803.65,1374,667.06,1702.19,...,Per Week,3,2,2,144.971027,-37.831809,206051512.0,loc9901d119afda,Melbourne,206051132.0
1,1,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,"['1 Bed', '1 Bath', '1 Parking']",Apartment / Unit / Flat,176,371.68,2265,352.49,2267.13,...,per week,1,1,1,144.953247,-37.816228,206041505.0,loc9901d119afda,Melbourne,206041122.0
2,2,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,"['1 Bed', '1 Bath']",Apartment / Unit / Flat,28,248.50,2189,401.73,1937.39,...,pw,1,1,0,144.958129,-37.812103,206041505.0,loc9901d119afda,Melbourne,206041122.0
3,3,3/27 Flinders Lane Melbourne VIC 3000,$420 per week,"['1 Bed', '1 Bath']",Apartment / Unit / Flat,140,404.26,2193,586.61,790.13,...,per week,1,1,0,144.973365,-37.814732,206041503.0,loc9901d119afda,Melbourne,206041122.0
4,4,611/408 Lonsdale Street Melbourne VIC 3000,$350,"['1 Bed', '1 Bath']",Apartment / Unit / Flat,28,525.08,2265,779.91,1657.95,...,NaN,1,1,0,144.960401,-37.812598,206041504.0,loc9901d119afda,Melbourne,206041122.0


In [4]:
# extract information needed
df1 = population[['SA2 code','ERP 2021']]

df2 = properties[['LOC_PID','SA2_CODE21']]
df2 = df2.fillna(0)
df2 = df2.drop_duplicates()
df2['SA2_CODE21']=df2['SA2_CODE21'].astype(int)

Find Overlap 


Some suburbs correspondes to multiple SA2s, find the area of each SA2 in the suburb.

In [5]:
suburb_shp = gpd.read_file("../data/raw/external_data/vic_localities/vic_localities.shp")
sa2_shp = gpd.read_file("../data/raw/external_data/SA2_shape/SA2_2021_AUST_GDA2020.shp")

In [6]:
# Suburb shape and area
suburb_shp = suburb_shp[['LOC_PID', 'LOC_NAME', 'geometry']].set_crs("EPSG:4326", allow_override=True)
suburb_shp['suburb_area'] = suburb_shp.geometry.area
suburb_area = suburb_shp.drop(columns='geometry')

# SA2 shape
sa2_shp = sa2_shp[sa2_shp['STE_NAME21'] == 'Victoria']
sa2_shp = sa2_shp[['SA2_CODE21', 'geometry']].set_crs("EPSG:4326", allow_override=True)
sa2_shp = sa2_shp.astype({'SA2_CODE21' : int})

/var/folders/td/8kxd5nlj15vc5q1zv2lhvlym0000gn/T/ipykernel_19625/1472629128.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  suburb_shp['suburb_area'] = suburb_shp.geometry.area


In [7]:
# find suburbs that belongs to multiple SA2
agg_suburb_count = df2.groupby('LOC_PID', as_index=False).count()
agg_suburb_count = agg_suburb_count[agg_suburb_count['SA2_CODE21'] > 1]

In [8]:
# Find overlap area
suburb_groups = df2.groupby('LOC_PID')
suburb_sa2_overlap_areas = pd.DataFrame([], columns=["LOC_PID", "overlap_area"])
for suburb in agg_suburb_count['LOC_PID']:
  group = suburb_groups.get_group(suburb )
  sub_poly = suburb_shp[suburb_shp['LOC_PID'] == suburb ]
  sa2_poly = sa2_shp[sa2_shp['SA2_CODE21'].isin(group['SA2_CODE21'].tolist())]
  overlap = gpd.overlay(sub_poly, sa2_poly, how="intersection")
  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)

/var/folders/td/8kxd5nlj15vc5q1zv2lhvlym0000gn/T/ipykernel_19625/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/td/8kxd5nlj15vc5q1zv2lhvlym0000gn/T/ipykernel_19625/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/td/8kxd5nlj15vc5q1zv2lhvlym0000gn/T/ipykernel_19625/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

In [9]:
suburb_sa2_overlap_areas  # The overlap_area [(SA2_code, area),...]

,LOC_PID,overlap_area
0,loc00a9769647d7,"[(207011522, 0.0004422559047194018), (20701152..."
1,loc00d1503504f1,"[(212051321, 0.0009383326105464119), (21205132..."
2,loc00f0949ea0ad,"[(210041541, 0.0070176802621758246), (21004154..."
3,loc0475beb19ff3,"[(212031557, 0.0007066527528316065), (21203155..."
4,loc0a03ed3531fd,"[(208031188, 0.00029753589690431127), (2080111..."
...,...,...
117,locf5fef268f8cd,"[(213051363, 0.00030836751688239234), (2130515..."
118,locf72490d7eaa6,"[(213051362, 0.0010877856749048145), (21305136..."
119,locfb463ae5ac4b,"[(212011546, 0.0010768714737548117), (21202155..."
120,locfd8472c41cbe,"[(211011256, 0.0008236757283031328), (21101125..."


Assign suburbs to SA2s

......

-----
### Find Suburb population proportion to SA2
1. Count the number of properties in suburbs
2. Use the properties proportion as the population proportion

In [10]:
result={}

In [12]:
# Find the LOC_NAME for each SA2
for sa in df1['SA2 code']:
    result[sa]={}
    a = list(set(df2.loc[df2['SA2_CODE21']==sa,:]['LOC_PID']))
    a_df = df2.loc[df2['SA2_CODE21']==sa,:]
    # The first type: one SA2 corresponds to multiple loc_NAME
    if len(a)>1:
        counts = 0
        counts_nums= []
        for locname in a:
            # The total number that each loc_name contains properties
            nums = a_df.loc[a_df['LOC_PID']==locname,:].shape[0]
            # Calculate the weight for each loc_name
            counts+=nums
            counts_nums.append(nums)
        ratio = list(map(lambda x:x/counts, counts_nums))
        # According to the population value of each SA2 multiplied by the weight of locality, the population of different locality is calculated.
        for l in range(len(a)):
            result[sa][a[l]] = ratio[l]*population.loc[population['SA2 code']==sa,:]['ERP 2021'].values[0]
            
    # The second type: A SA2 corresponds to a loc_name
    elif len(a)==1:
        # Just give the value of the population in sa2 directly to that locality
        value = population.loc[population['SA2 code']==sa,:]['ERP 2021'].values[0]
        result[sa][a[0]]=value    

In [13]:
result1 = {}
locality = list(set(df2['LOC_PID']))
for loc in locality:
    # The third type: Multiple SA2 correspond to one locality.
    b = df2.loc[df2['LOC_PID']==loc,:]
    b = b.drop_duplicates()
    if b.shape[0]>1:
        result1[loc]={}
        # The population of this loc_name is equal to the sum of the populations of SA2 belonging to this locality
        code = list(b['SA2_CODE21'])
        v = 0
        for c in code:
            value = population.loc[population['SA2 code']==c,:]['ERP 2021'].values[0]
            v+=value
        result1[loc]=v

IndexError: index 0 is out of bounds for axis 0 with size 0

------
### Predict Suburb Future Population and Density


In [ ]:
result

{201011001: {'Lucas': 5004.0, 'Cardigan': 417.0, 'Alfredton': 10842.0},
 201011002: {'Lake Gardens': 1491.375,
  'Ballarat Central': 5592.65625,
  'Newington': 1491.375,
  'Lake Wendouree': 2609.90625,
  'Bakery Hill': 745.6875},
 201011003: {'Brown Hill': 5229.2307692307695,
  'Nerrina': 475.3846153846154,
  'Ballarat East': 5704.615384615385,
  'Black Hill': 4753.846153846154,
  'Invermay Park': 475.3846153846154,
  'Ballarat North': 2852.3076923076924,
  'Soldiers Hill': 4278.461538461538,
  'Eureka': 950.7692307692308},
 201011004: {'Canadian': 7298.0,
  'Mount Clear': 1824.5,
  'Golden Point': 2280.625,
  'Sebastopol': 7298.0,
  'Mount Pleasant': 3192.875,
  'Redan': 3649.0},
 201011005: {'Mount Helen': 5112.0, 'Buninyong': 2556.0},
 201011006: {'Winter Valley': 6140.659090909091,
  'Delacombe': 1364.590909090909,
  'Bonshaw': 1592.0227272727273,
  'Smythes Creek': 909.7272727272727},
 201011007: {},
 201011008: {'Miners Rest': 1383.2727272727273,
  'Wendouree': 13141.09090909091,

In [ ]:
result1

{'Ringwood East': 26483,
 'Malvern East': 44604,
 'Berwick': 53459,
 'Boronia': 36875,
 'Coburg North': 22354,
 'Deanside': 42411,
 'Warrnambool': 35957,
 'Sandringham': 30783,
 'Cheltenham': 37801,
 'Armadale': 31604,
 'Greensborough': 30972,
 'Surrey Hills': 48984,
 'Templestowe Lower': 26053,
 'Murrumbeena': 30144,
 'Hadfield': 57697,
 'Mount Waverley': 36010,
 'St Kilda': 47139,
 'Flemington': 10777,
 'Pascoe Vale South': 66818,
 'Pascoe Vale': 35852,
 'Fitzroy North': 27999,
 'Point Cook': 80064,
 'Macleod': 31920,
 'St Kilda East': 41212,
 'St Albans': 38490,
 'Clyde': 76241,
 'Brunswick West': 44931,
 'Melbourne': 210570,
 'Camberwell': 72921,
 'Ferntree Gully': 29782,
 'Bundoora': 59356,
 'Narre Warren': 28743,
 'Sunbury': 45899,
 'Mont Albert': 26466,
 'Mill Park': 30025,
 'Shepparton': 44215,
 'Southbank': 39616,
 'Oakleigh South': 35995,
 'Truganina': 52835,
 'Glen Waverley': 41928,
 'Glen Iris': 38489,
 'Ormond': 30989,
 'Officer': 46678,
 'Noble Park': 32491,
 'Narre Warre

In [ ]:
pd.DataFrame(result)

,201011001,201011002,201011003,201011004,201011005,201011006,201011007,201011008,201021009,201021010,...,217031471,217031472,217031473,217031474,217031475,217031476,217041477,217041478,217041479,217041480
loc82f5b45a790d,10842.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lake Gardens,NaN,1491.37500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ballarat Central,NaN,5592.65625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Port Fairy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4988.50,NaN,NaN
locf00a01fc543b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2494.25,NaN,NaN
Hawkesdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2494.25,NaN,NaN
loc1f73672977ce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22586.0,12584.470588
loc60566a23e081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,786.529412


In [ ]:
df = pd.DataFrame(result)
r2 = df.sum(axis=1)
r = pd.DataFrame(r2)
r.to_csv("../data/raw/result1.csv")


In [ ]:
r1 = pd.DataFrame.from_dict(result1,orient='index')
r1.to_csv("../data/raw/Multiple_SA2_correspond_to_one_locality.csv")

In [ ]:
r1

,0
Ringwood East,26483
Malvern East,44604
Berwick,53459
Boronia,36875
Coburg North,22354
...,...
Brighton East,35781
Hawthorn,41782
Highett,37801
Fawkner,31288
